In [0]:
# Required packages for this lab
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import time

In [0]:
# Import dataset from csv file
df = pd.read_csv('/dbfs/FileStore/ConcurrentModelTraining/data.csv',parse_dates=['date'], header='infer')
df.head(5)

Out[2]:

,date,free_space_gb,disk_id
0,2021-06-01,53.12,0
1,2021-06-02,51.41,0
2,2021-06-03,51.45,0
3,2021-06-04,47.81,0
4,2021-06-05,48.30,0


In [0]:
from pyspark.sql.types import *

schema_log_time = StructType([
    StructField("method",StringType(),True),
    StructField("duration",FloatType(),True)
])

df_log_time = spark.createDataFrame(sc.emptyRDD(), schema_log_time)

In [0]:
# Create our Linear Regression function
# Split the data into train/test data
# Predict the future usage
# Return DataFrame to visualise our predictions 
# Calculate the RMSE

def regression(df:pd.DataFrame) -> pd.DataFrame:
    df.reset_index(drop=True,inplace=True)
    X = df.index.values.reshape(-1,1)
    y = df.iloc[:, 1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,shuffle=False)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    date_pred = df[df.index>=X_test.min()].date

    df_full = df.copy()
    df_full['prediction'] = [None] * X_test.min() + list(y_pred)

    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    result = [df_full, rmse]
    return result

In [0]:
# Sequential predictions
start_time = time.perf_counter()

# Create output DataFrames
df_sequential_prediction = pd.DataFrame(columns = ['date','free_space_gb','disk_id','prediction'])
df_sequential_rmse = pd.DataFrame(columns = ['disk_id','rmse'])

for c in df['disk_id'].unique():
    df_each = df.loc[df['disk_id'] == c]
    df_each_prediction, rmse = regression(df_each)
    df_sequential_prediction = df_sequential_prediction.append(df_each_prediction, ignore_index=True)
    df_sequential_rmse = df_sequential_rmse.append({'disk_id': c,'rmse':rmse}, ignore_index=True)
duration = time.perf_counter() - start_time
print("--- {} seconds ---".format(duration))

--- 4096.903429766 seconds ---

In [0]:
rows = [['sequential', duration]]
columns = ['method', 'duration']
df_sequential_time = spark.createDataFrame(rows, columns)
 
# Add this to our DataFrame
df_log_time = df_log_time.union(df_sequential_time)

df_log_time.show()

+----------+--------------+
 method| duration|
+----------+--------------+
sequential|4096.903429766|
+----------+--------------+

In [0]:
df_sequential_prediction

Out[7]:

,date,free_space_gb,disk_id,prediction
0,2021-06-01,53.12,0,NaN
1,2021-06-02,51.41,0,NaN
2,2021-06-03,51.45,0,NaN
3,2021-06-04,47.81,0,NaN
4,2021-06-05,48.30,0,NaN
...,...,...,...,...
29995,2021-06-26,27.30,999,24.800059
29996,2021-06-27,24.69,999,23.782872
29997,2021-06-28,21.83,999,22.765686
29998,2021-06-29,20.25,999,21.748499


In [0]:
# Parallel processing with concurrent.futures
# CPU intensive process, so will look to use ProcessPoolExecutor 
import concurrent.futures

# Create DataFrame of DataFrames to input into concurrent futures
disk_dfs = []
disks = df['disk_id'].unique()
for i in disks:
    disk_dfs.append(df[df.disk_id == i].copy())

# Create output DataFrames
df_concurrent_prediction = pd.DataFrame(columns = ['date','free_space_gb','disk_id','prediction'])
df_concurrent_rmse = pd.DataFrame(columns = ['disk_id','rmse'])

start_time = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    future = list(map(lambda x: executor.submit(regression, x),disk_dfs))
    finished,unfinished = concurrent.futures.wait(future)
    for x in finished:
        try:
            df_concurrent_prediction=df_concurrent_prediction.append(x.result()[0])
            disk_id = x.result()[0].disk_id[0]
            rmse = x.result()[1]
            new_row = {'disk_id': disk_id,'rmse':rmse}
            df_concurrent_rmse = df_concurrent_rmse.append(new_row, ignore_index=True)
        except Exception as e:
            print(e,type(e))
            
duration_concurrent = time.perf_counter() - start_time
print("--- {} seconds ---".format(duration_concurrent))

--- 9.886265507000644 seconds ---

In [0]:
rows = [['concurrent.futures', duration_concurrent]]
columns = ['method', 'duration']
df_concurrent_time = spark.createDataFrame(rows, columns)

# Add this to our DataFrame
df_log_time = df_log_time.union(df_concurrent_time)

df_log_time.show()

+------------------+-----------------+
 method| duration|
+------------------+-----------------+
 sequential| 4096.903429766|
concurrent.futures|9.886265507000644|
+------------------+-----------------+

In [0]:
 df_concurrent_prediction

Out[10]:

,date,free_space_gb,disk_id,prediction
0,2021-06-01,44.88,67,NaN
1,2021-06-02,46.70,67,NaN
2,2021-06-03,45.64,67,NaN
3,2021-06-04,44.07,67,NaN
4,2021-06-05,39.39,67,NaN
...,...,...,...,...
25,2021-06-26,24.41,285,25.064345
26,2021-06-27,25.54,285,24.020284
27,2021-06-28,25.31,285,22.976223
28,2021-06-29,27.28,285,21.932162


In [0]:
# Create Spark DataFrame of csv file
spark_df = spark.read \
    .option("header",True) \
    .option("inferSchema","True") \
    .csv("dbfs:/FileStore/ConcurrentModelTraining/data.csv")

In [0]:
from pyspark.sql.types import *

# Define schema of output results
result_schema =StructType([
  StructField('date',DateType()),
  StructField('free_space_gb',FloatType()),
  StructField('disk_id',IntegerType()),
  StructField('prediction',FloatType())
])

# Define schema of evaluation results
evaluation_schema =StructType([
  StructField('disk_id',IntegerType()),
  StructField('rmse',FloatType())
])


In [0]:
# Create new regression function for Pandas UDFs
# No longer need to specify the @pandas_udf(df.schema, PandasUDFType.GROUPED_MAP) decorator since Apache Spark 3.0:
# https://databricks.com/blog/2020/05/20/new-pandas-udfs-and-python-type-hints-in-the-upcoming-release-of-apache-spark-3-0.html
def regression_udf(df: pd.DataFrame) -> pd.DataFrame:
    df.reset_index(drop=True,inplace=True)
    X = df.index.values.reshape(-1,1)
    y = df.iloc[:, 1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,shuffle=False)
    
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    y_pred = regressor.predict(X_test)
    date_pred = df[df.index>=X_test.min()].date
    
    df_full = df.copy()
    df_full['prediction'] = [None] * X_test.min() + list(y_pred)

    return df_full

# Separate out RMSE function 
def evaluation_rmse(df: pd.DataFrame) -> pd.DataFrame:
    disk = df.disk_id.iloc[0]
    rmse = np.sqrt(metrics.mean_squared_error(df.dropna().data, df.dropna().prediction))

    results = {'disk_id':[disk], 'rmse':[rmse]}
    return pd.DataFrame.from_dict( results )

In [0]:
# Parallel executions with Pandas UDFs
start_time = time.perf_counter()

# Create output DataFrames
df_udfs_prediction = spark_df.groupBy('disk_id').applyInPandas(regression_udf, result_schema)
df_udfs_rmse = df_udfs_prediction.groupBy('disk_id').applyInPandas(evaluation_rmse, evaluation_schema)

duration_udfs = time.perf_counter() - start_time
print("--- {} seconds ---".format(duration_udfs))

--- 0.13279857599991374 seconds ---

In [0]:
df_udfs_prediction.show(100)

+----------+-------------+-------+-----------+
 date|free_space_gb|disk_id| prediction|
+----------+-------------+-------+-----------+
2021-06-01| 53.12| 0| null|
2021-06-02| 51.41| 0| null|
2021-06-03| 51.45| 0| null|
2021-06-04| 47.81| 0| null|
2021-06-05| 48.3| 0| null|
2021-06-06| 43.44| 0| null|
2021-06-07| 44.46| 0| null|
2021-06-08| 43.91| 0| null|
2021-06-09| 40.32| 0| null|
2021-06-10| 40.42| 0| null|
2021-06-11| 40.52| 0| null|
2021-06-12| 41.53| 0| null|
2021-06-13| 36.27| 0| null|
2021-06-14| 40.05| 0| null|
2021-06-15| 37.56| 0| null|
2021-06-16| 34.72| 0| null|
2021-06-17| 32.75| 0| null|
2021-06-18| 37.42| 0| null|
2021-06-19| 36.03| 0| null|
2021-06-20| 31.32| 0| null|
2021-06-21| 32.14| 0| null|
2021-06-22| 31.64| 0| null|
2021-06-23| 28.78| 0| null|
2021-06-24| 29.61| 0| null|
2021-06-25| 29.01| 0| 27.57442|
2021-06-26| 25.68| 0| 26.597107|
2021-06-27| 28.25| 0| 25.619795|
2021-06-28| 22.7| 0| 24.64248|
2021-06-29| 21.78| 0| 23.665169|
2021-06-30| 21.02| 0| 22.687855|
2021-06-01| 44.78| 1| null|
2021-06-02| 46.45| 1| null|
2021-06-03| 42.65| 1| null|
2021-06-04| 45.0| 1| null|
2021-06-05| 42.75| 1| null|
2021-06-06| 43.38| 1| null|
2021-06-07| 40.42| 1| null|
2021-06-08| 40.96| 1| null|
2021-06-09| 36.53| 1| null|
2021-06-10| 34.85| 1| null|
2021-06-11| 35.74| 1| null|
2021-06-12| 35.46| 1| null|
2021-06-13| 33.62| 1| null|
2021-06-14| 32.02| 1| null|
2021-06-15| 27.77| 1| null|
2021-06-16| 29.93| 1| null|
2021-06-17| 26.09| 1| null|
2021-06-18| 29.13| 1| null|
2021-06-19| 25.27| 1| null|
2021-06-20| 27.1| 1| null|
2021-06-21| 26.16| 1| null|
2021-06-22| 23.27| 1| null|
2021-06-23| 20.96| 1| null|
2021-06-24| 20.91| 1| null|
2021-06-25| 19.58| 1| 19.693478|
2021-06-26| 22.95| 1| 18.564957|
2021-06-27| 18.02| 1| 17.436436|
2021-06-28| 22.0| 1| 16.307913|
2021-06-29| 19.27| 1| 15.179391|
2021-06-30| 15.82| 1| 14.05087|
2021-06-01| 51.78| 2| null|
2021-06-02| 50.76| 2| null|
2021-06-03| 45.7| 2| null|
2021-06-04| 46.93| 2| null|
2021-06-05| 43.44| 2| null|
2021-06-06| 43.25| 2| null|
2021-06-07| 42.39| 2| null|
2021-06-08| 36.61| 2| null|
2021-06-09| 32.61| 2| null|
2021-06-10| 36.52| 2| null|
2021-06-11| 31.37| 2| null|
2021-06-12| 30.76| 2| null|
2021-06-13| 26.18| 2| null|
2021-06-14| 22.36| 2| null|
2021-06-15| 21.32| 2| null|
2021-06-16| 18.26| 2| null|
2021-06-17| 20.51| 2| null|
2021-06-18| 18.71| 2| null|
2021-06-19| 16.48| 2| null|
2021-06-20| 14.73| 2| null|
2021-06-21| 12.66| 2| null|
2021-06-22| 11.72| 2| null|
2021-06-23| 7.45| 2| null|
2021-06-24| 5.73| 2| null|
2021-06-25| 4.53| 2| 3.7361956|
2021-06-26| 5.2| 2| 1.7409914|
2021-06-27| 4.74| 2|-0.25421304|
2021-06-28| 2.27| 2| -2.2494173|
2021-06-29| 2.03| 2| -4.2446218|
2021-06-30| -0.88| 2| -6.239826|
2021-06-01| 50.24| 3| null|
2021-06-02| 47.62| 3| null|
2021-06-03| 47.26| 3| null|
2021-06-04| 44.16| 3| null|
2021-06-05| 49.14| 3| null|
2021-06-06| 46.64| 3| null|
2021-06-07| 46.25| 3| null|
2021-06-08| 47.76| 3| null|
2021-06-09| 42.65| 3| null|
2021-06-10| 41.23| 3| null|
+----------+-------------+-------+-----------+
only showing top 100 rows

In [0]:
rows = [['pandas UDFs', duration_udfs]]
columns = ['method', 'duration']
df_udfs_time = spark.createDataFrame(rows, columns)

# Add this to our DataFrame
df_log_time = df_log_time.union(df_udfs_time)

df_log_time.show()

+------------------+-------------------+
 method| duration|
+------------------+-------------------+
 sequential| 4096.903429766|
concurrent.futures| 9.886265507000644|
 pandas UDFs|0.13279857599991374|
+------------------+-------------------+

In [0]:
# Final step: add to SQL table and create visuals.
# df_log_time.createOrReplaceTempView("methods")
df_log_time.write.saveAsTable("methods")

In [0]:
%sql
select 
  method, 
  duration, 
  round(100 * duration / (select max(duration) from methods), 3) as percent_of_max_duration,
  round((select max(duration) from methods) / duration) as times_faster_than_slowest
from methods
order by percent_of_max_duration desc

method,duration,percent_of_max_duration,times_faster_than_slowest
sequential,4096.903429766,100.0,1.0
concurrent.futures,9.886265507000644,0.241,414.0
pandas UDFs,0.13279857599991374,0.003,30851.0


In [0]:
%sql
-- Logarithmic scale to visualise the durations
select method, log(duration) as log_duration
from methods
order by log_duration desc

method,log_duration
sequential,8.317986706306415
concurrent.futures,2.291146471392492
pandas UDFs,-2.01892176488951


In [0]:
# Final step: add to SQL table and create visuals.
# df_udfs_prediction.createOrReplaceTempView("predictions")
df_udfs_prediction.write.saveAsTable("predictions")

In [0]:
%sql
-- Plot graph of predictions vs actuals
select * 
from predictions
where disk_id < 3 -- As to not clutter the graph

date,free_space_gb,disk_id,prediction
2021-06-01,53.12,0,null
2021-06-02,51.41,0,null
2021-06-03,51.45,0,null
2021-06-04,47.81,0,null
2021-06-05,48.3,0,null
2021-06-06,43.44,0,null
2021-06-07,44.46,0,null
2021-06-08,43.91,0,null
2021-06-09,40.32,0,null
2021-06-10,40.42,0,null
